In [14]:
import pandas as pd
import altair as alt
import panel as pn
pn.extension('vega')



In [15]:
# standardizing state and county column names

apple_counties = pd.read_csv('data/apple_clean_counties.csv', parse_dates = ['date'])
google_counties = pd.read_csv('data/google_clean_counties.csv', parse_dates = ['date'])

apple_counties['state'] = apple_counties['sub_region']
apple_counties['county'] = apple_counties['region']

google_counties['state'] = google_counties['sub_region_1']
google_counties['county'] = google_counties['sub_region_2']

state_county_combinations = []
for i in list(zip(apple_counties.state, apple_counties.county)):
    state_county_combinations.append(', '.join(i))

state_county_combinations = set(state_county_combinations)   

case_data = pd.read_csv('data/jhu-case-data.csv', parse_dates= ['date'])

In [22]:
state_county_combinations
case_data.head(3)

,Unnamed: 0,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,date,cases
0,0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,"American Samoa, US",2020-01-22,0
1,1,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,"Guam, US",2020-01-22,0
2,2,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,"Northern Mariana Islands, US",2020-01-22,0


In [44]:

display(apple_counties[(apple_counties.state == 'Maryland') & (apple_counties.county == 'Calvert County')])

display(apple_counties[(apple_counties.state == 'Maryland') & (apple_counties.county == 'Calvert County')].melt(
        id_vars='date',
        value_vars = ['driving', 'transit', 'walking'],
        var_name = 'type',
        value_name = 'volume'
    ))

,date,region,sub_region,country,geo_type,driving,transit,walking,state,county
245,2020-01-13,Calvert County,Maryland,United States,county,100.00,NaN,NaN,Maryland,Calvert County
2335,2020-01-14,Calvert County,Maryland,United States,county,102.48,NaN,NaN,Maryland,Calvert County
4425,2020-01-15,Calvert County,Maryland,United States,county,110.35,NaN,NaN,Maryland,Calvert County
6515,2020-01-16,Calvert County,Maryland,United States,county,106.16,NaN,NaN,Maryland,Calvert County
8605,2020-01-17,Calvert County,Maryland,United States,county,123.26,NaN,NaN,Maryland,Calvert County
...,...,...,...,...,...,...,...,...,...,...
261495,2020-05-17,Calvert County,Maryland,United States,county,121.86,NaN,NaN,Maryland,Calvert County
263585,2020-05-18,Calvert County,Maryland,United States,county,123.09,NaN,NaN,Maryland,Calvert County
265675,2020-05-19,Calvert County,Maryland,United States,county,127.85,NaN,NaN,Maryland,Calvert County
267765,2020-05-20,Calvert County,Maryland,United States,county,129.20,NaN,NaN,Maryland,Calvert County


,date,type,volume
0,2020-01-13,driving,100.00
1,2020-01-14,driving,102.48
2,2020-01-15,driving,110.35
3,2020-01-16,driving,106.16
4,2020-01-17,driving,123.26
...,...,...,...
385,2020-05-17,walking,NaN
386,2020-05-18,walking,NaN
387,2020-05-19,walking,NaN
388,2020-05-20,walking,NaN


In [36]:
# state_input = pn.widgets.AutocompleteInput(name = 'Select a state, county',
#                                         options = list(state_county_combinations),
#                                        placeholder = 'ex: Maryland, Calvert County',
#                                           value = 'Maryland, Calvert County')

state_input = pn.widgets.Select(name = 'Select a state, county',
                                        options = list(state_county_combinations),
                                       placeholder = 'ex: Maryland, Calvert County',
                                          value = 'Maryland, Calvert County')

@pn.depends(state_input.param.value)
def state_county_plot(state_input):
    state, county = state_input.split(', ')

    apple_to_plot_sc = apple_counties[(apple_counties.state == state) & (apple_counties.county == county)]
    google_to_plot_sc = google_counties[(google_counties.state == state) & (google_counties.county == county)]

    apple_to_plot_sc = apple_to_plot_sc.melt(
        id_vars='date',
        value_vars = ['driving', 'transit', 'walking'],
        var_name = 'type',
        value_name = 'volume'
    )

    google_cols_to_melt_sc = google_to_plot_sc.columns[4:]
    google_to_plot_sc = google_to_plot_sc.melt(
        id_vars = 'date',
        value_vars = google_cols_to_melt_sc,
        var_name = 'type',
        value_name = 'volume'
    )


    brush = alt.selection_interval(encodings=['x'])  

    color = alt.condition(brush,
                          alt.Color('type:Q', legend=None),
                          alt.value('lightgray'))


    apple_sc = apple = alt.Chart(apple_to_plot_sc).mark_line().encode(
        x = 'date:T',
        y = 'volume:Q',
        color = 'type:N'
    ).add_selection(brush).properties(title = 'Apple mobility data')

    google_sc = google = alt.Chart(google_to_plot_sc).mark_line().encode(
        x = alt.X('date:T', scale = alt.Scale(domain = brush)),
        y = 'volume:Q',
        color = 'type:N'
    ).properties(title = 'Google mobility data')
    
    subtitle = f"### Mobility and case data for {state}, {county}"
    
    
    county_to_plot = pd.DataFrame(
        case_data[(case_data.Province_State == state) & (case_data.Admin2 == county.split()[0])]
    )
    county_to_plot['new_cases'] = county_to_plot['cases'].rolling(window=2).apply(lambda x: x[1] - x[0], raw = True)
    
    
    county_cum_cases = alt.Chart(county_to_plot).mark_line().encode(
        x = alt.X('date:T', scale = alt.Scale(domain = brush), title = 'Cumulative Cases'),
        y = 'cases:Q'
    ).properties(
        title = {'text': 'Daily New Cases', 
                    'subtitle': 'Source: JHU'}
    )

    county_new_cases = alt.Chart(county_to_plot).mark_line().encode(
        x = alt.X('date:T', scale = alt.Scale(domain = brush), title = 'Daily New Cases'),
        y = 'new_cases:Q'
    ).properties(
        title = {'text': 'Daily New Cases', 
                    'subtitle': 'Source: JHU'}
    )


    county_plots_set = alt.vconcat(apple_sc | google_sc, county_cum_cases | county_new_cases)
    
    return pn.Column(subtitle, county_plots_set)



state_county_dash = pn.Row(
    pn.Column(state_input,state_county_plot)
)


In [37]:
# state_input.value = 'Indiana, Lake County'
state_county_dash


Row
    [0] Column
        [0] Select(options=['Indiana, Lake County', ...], value='Maryland, Calvert C...)
        [1] ParamFunction(function, _pane=Column)

In [40]:
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                   'B': {0: 1, 1: 3, 2: 5},
                   'C': {0: 2, 1: 4, 2: 6}})
df
#    A  B  C
# 0  a  1  2
# 1  b  3  4
# 2  c  5  6

,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6
